# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Assignment 7: Decision Trees

---
** Setup **

In [10]:
require 'test/unit/assertions'
require 'daru'
require 'distribution'
require 'json'

include Test::Unit::Assertions

## Loads data files
def read_sparse_data_from_csv prefix
  data = []
  classes = Hash.new {|h,k| h[k] = 0}
  header = File.read(prefix + ".header").chomp.split(",")  
  
  File.open(prefix + ".csv").each_line.with_index do |l, i|
    a = l.chomp.split ","
    next if a.empty?
    row = {"features" => Hash.new}
    
    header.each.with_index do |k, i|
      v = a[i].to_f
      if k == "label"
        row["label"] = v.to_i
      else
        next if v.zero?
        row["features"][k] = v
      end
    end
    classes[row["label"]] += 1
    
    data << row
  end
  return {"classes" => classes, "features" => header[0,header.size - 1], "data" => data}
end

:read_sparse_data_from_csv

In [11]:
#Preview 2 lines from the Iris dataset
iris = read_sparse_data_from_csv "iris"
iris["data"][0,2]

[{"features"=>{"sepal_length"=>7.0, "sepal_width"=>3.2, "petal_length"=>4.7, "petal_width"=>1.4}, "label"=>1}, {"features"=>{"sepal_length"=>5.7, "sepal_width"=>3.0, "petal_length"=>4.2, "petal_width"=>1.2}, "label"=>1}]

In [12]:
#Preview 2 lines from the Spambase dataset
spambase = read_sparse_data_from_csv "spambase"
spambase["data"][0,2]

[{"features"=>{"word_freq_our"=>0.27, "word_freq_mail"=>0.83, "word_freq_you"=>0.27, "word_freq_your"=>0.27, "word_freq_font"=>8.58, "char_freq_["=>0.092, "char_freq_$"=>0.185, "char_freq_#"=>0.232, "capital_run_length_average"=>7.313, "capital_run_length_longest"=>99.0, "capital_run_length_total"=>607.0}, "label"=>1}, {"features"=>{"word_freq_your"=>0.9, "word_freq_george"=>0.9, "word_freq_data"=>0.9, "char_freq_["=>0.14, "capital_run_length_average"=>3.472, "capital_run_length_longest"=>28.0, "capital_run_length_total"=>125.0}, "label"=>0}]

---
## Question 1.1 (10 Points)
Write a function that calculates the class distribution, $p(c)$, for all classes $c$ for a dataset. 

In [13]:
def class_distribution dataset
  # BEGIN YOUR CODE
    prob = Hash.new()
    if dataset.nil?
    else 
      i = 0
      distribution = []
      while i< dataset.length do
        distribution.append(dataset[i]["label"])
        i+=1
      end
      array = distribution.sort 
      type = array.select{ |x| array.count(x) >= 1}.uniq
      type.each do |n|
        prob.store(n,distribution.count(n)/(dataset.length).to_f)
      end
    end
    return prob
  
  
  
  #END YOUR CODE
end

:class_distribution

In [14]:
### TESTS ###
# Check that there are three classes
t1_iris_dist = class_distribution iris["data"]
t1_iris_num_classes = 3
assert_equal(t1_iris_num_classes, t1_iris_dist.size)

In [15]:
### TESTS ###
# Check that each class has a probability 1/3
t1_iris_dist = class_distribution iris["data"]
t1_iris_num_classes = 3
n = t1_iris_num_classes.times do |cls|
  assert_in_delta t1_iris_dist[cls], 0.33333, 1e-4
end
assert_equal t1_iris_num_classes, n

In [16]:
### TESTS ###
t1_spambase_dist = class_distribution spambase["data"]
t1_spambase_num_classes = 2
assert_equal(t1_spambase_num_classes, t1_spambase_dist.size)

In [17]:
### TESTS ###
t1_spambase_dist = class_distribution spambase["data"]
assert_in_delta t1_spambase_dist[0], 0.6, 1e-1
assert_in_delta t1_spambase_dist[1], 0.4, 1e-1

## Question 2 (20 Points)
Adapting your code for calculating the entropy from a previous assignment, use input from a hash table rather than an array. 

In [18]:
def entropy dist
  # BEGIN YOUR CODE  
  if dist.empty?
    en = 0
  else
    arr =(dist.to_a).transpose[1]
    if arr.include? 0 or arr.empty?
      en = 0
    else
      en = arr.inject(0.0){|sum,x| sum -x*Math.log(x) }
    end
  end
return en
  #END YOUR CODE
end

:entropy

In [19]:
### TESTS ###
#Checks the class entropy for Iris dataset
t2_iris_dist = class_distribution iris["data"]
t2_iris_entropy = entropy t2_iris_dist
assert_in_delta(1.0986122886681096, t2_iris_entropy, 1e-4)

In [20]:
### TESTS ###
#Checks the class entropy for Spambase dataset
t2_spambase_dist = class_distribution spambase["data"]
t2_spambase_entropy = entropy t2_spambase_dist
assert_in_delta(0.6705230209876485, t2_spambase_entropy, 1e-4)

In [21]:
### TESTS ###
# Checks that code handles a class with zero members of a class
t2_zero_dist = {0 => 0.0, 1 => 107.0}
t2_zero_entropy = entropy t2_zero_dist
assert_in_delta(0.0, t2_zero_entropy, 1e-4)

## Question 3.1 (5 points)

Implement information gain given an initial entropy $H_0$ and a hash of counts, where the key of each hash is a split criterion and the value is an array of all examples matching the split. Call the ```class_distribution``` and ```entropy``` functions you have defined above.

The formula for information gain is as follows:

## $IG(Q, V) = H_0 - \sum_{v\in V} \frac{|v|}{|V|} H(c\mid v) $ 

In [22]:
def information_gain h0, splits
  # BEGIN YOUR CODE


  i = 0
  name = splits.keys
  sum =0.0
  h = 0.0
  while i<splits.length do
    h += (splits[name[i]].length) * entropy( class_distribution(splits[name[i]]))
    sum += splits[name[i]].length
    i+=1
  end
  return h0- h/sum
  
  #END YOUR CODE
end

:information_gain

In [23]:
### TEST: Random Split ###
t3_random_split = iris["data"].group_by {|row| rand > 0.5 ? "l" : "r"}
t3_entropy = entropy(class_distribution(iris["data"]))

assert_true(!t3_entropy.nil?)

## Question 3.2 (5 points)

The test above verifies the information gain when the iris data is split with uniform probability. What should the information gain be here if we were to do this for the **spambase** dataset?

In [24]:
## Make this function return a constant equal to your answer to the question above.
def t3_random_information_gain_spambase
  # BEGIN YOUR CODE
  spambase = read_sparse_data_from_csv "spambase"
  t3_random_split = spambase["data"].group_by {|row| rand > 0.5 ? "l" : "r"}
  t3_entropy = entropy(class_distribution(spambase["data"]))
  return information_gain t3_entropy, t3_random_split
  #END YOUR CODE
end

:t3_random_information_gain_spambase

In [25]:
### TEST (Hidden) ###
assert_true(t3_random_information_gain_spambase() >= 0)

## Question 3.3 (10 points)

The test below verifies the information gain of the iris data when split by class value. What should the information gain be here if we were to do this for the **spambase** dataset?

In [26]:
### TEST: Perfect split (by class)
t3_perfect_split = iris["data"].group_by {|row| row["label"]}
t3_perfect_information_gain = information_gain t3_entropy, t3_perfect_split
assert_true(!t3_perfect_information_gain.nil?)


In [27]:
## Make this function return a constant equal to your answer to the question above.
def t3_perfect_information_gain_spambase
  # BEGIN YOUR CODE
  spambase = read_sparse_data_from_csv "spambase"
  t3_perfect_split = spambase["data"].group_by {|row|  row["label"]}
  t3_entropy = entropy(class_distribution(spambase["data"]))
  return information_gain t3_entropy, t3_perfect_split
  
  
  
  #END YOUR CODE
end

:t3_perfect_information_gain_spambase

In [28]:
### TEST (Hidden) ###
assert_true (t3_perfect_information_gain_spambase() > 0) 

## Question 4 (20 points)
Considering the only the figure below, which shows the iris dataset colored by class, what value of petal_length would have the best information gain?

In [29]:
d = iris["data"]
df = Daru::DataFrame.new({
  x1: d.collect{|r| r["features"]["petal_length"]}, 
  x2: d.collect{|r| r["features"]["sepal_width"]}, 
  label: d.collect{|r| r["label"]}
}).to_category(:label).plot(type: :scatter, x: :x1, y: :x2, categorized: {by: :label, method: :color}) do |plot, diagram|
  plot.x_label iris["features"][2]
  plot.y_label iris["features"][1]
  plot.legend true
end

#<Nyaplot::Plot:0x0000561e1efeb4c8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000561e1efb67c8 @properties={:type=>:scatter, :options=>{:x=>:x1, :y=>:x2, :title=>1, :color=>"rgb(102,194,165)", :tooltip_contents=>[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, :data=>"98e546cb-2818-44a4-9b87-7d10aafb50dd"}, @xrange=[3.0, 5.1], @yrange=[2.0, 3.4]>, #<Nyaplot::Diagram:0x0000561e1ef84ea8 @properties={:type=>:scatter, :options=>{:x=>:x1, :y=>:x2, :title=>2, :color=>"rgb(252,141,98)", :tooltip_contents=>[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]}, :data=>"f7be8237-7f27-410e-af56-faa963fe23f5"}, @xrange=[4.5, 6.9], @yrange=[2.2, 3.8]>, #<Nyaplot::Diagram:0x0000561e1ebd4df8 @properties={:type=>:scatter, :options=>{:x=>:x1, :y=>:x2, :title=>0, :color=>"rgb(141,160,203)", :tooltip_contents=>[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, :data=>"34381051-770a-47c0-a60c-35111d386632"}, @xrange=[1.0, 1.9], @yrange=[2.3, 4.4]>], :options=>{:legend=>true, :x_label=>"petal_length", :y_label=>"sepal_width", :zoom=>true, :width=>800, :xrange=>[1.0, 6.9], :yrange=>[2.0, 4.4]}}>

In [30]:
## Make this function return a constant equal to your answer to the question above.
def t4_best_split_for_petal_length_guess
  # BEGIN YOUR CODE

  return 2.4
  #END YOUR CODE
end

:t4_best_split_for_petal_length_guess

In [31]:
### TEST (Hidden) ###

## Question 5 (20 points)
Implement a function which takes a feature name and a real-valued threshold and splits data into two groups: 
* Strictly less than the threshold
* Greater than or equal to the threholds. 

This function returns a structure that can be used to calculate the information gain. An example is as follows:
```
{"petal_length < X1":[
        {"features":{"sepal_length":4.8,"sepal_width":3.1,"petal_length":1.6,"petal_width":0.2},"label":0},
],"petal_length >= X1":[
        {"features":{"sepal_length":4.8,"sepal_width":3.1,"petal_length":1.6,"petal_width":0.2},"label":0},
]}
```

In [32]:
def split_on_numeric_value x, k, v
  # BEGIN YOUR CODE
  i = 0
  split_new = Hash.new()
  list1 = []
  list2 = []

  while i<x.length do
    if x[i]["features"].include? k
      if x[i]["features"][k] < v 
          list1.append(x[i])
      elsif x[i]["features"][k] >= v
          list2.append(x[i])
      end
    else
       list1.append(x[i])
    end
    i+=1
  end
  split_new.store(k + " < " + v.to_s,list1)
  split_new.store(k + " >= " + v.to_s,list2)

  return split_new
  #END YOUR CODE
end

:split_on_numeric_value

In [33]:
### TEST ###

# Checks for the number of examples when split on petal_length = 1.7
t5_iris_splits = split_on_numeric_value iris["data"], "petal_length", 1.7
t5_split_sizes = t5_iris_splits.values.collect {|v| v.size}.sort
t5_num_splits = 2
assert_equal t5_num_splits, t5_split_sizes.size
assert_equal 44, t5_split_sizes[0]
assert_equal 106, t5_split_sizes[1]

In [34]:
### TEST ###

# Checks the information gain for this split
t5_iris_entropy = entropy(class_distribution(iris["data"]))
t5_iris_information_gain = information_gain t5_iris_entropy, t5_iris_splits
assert_in_delta 0.48280104455013506, t5_iris_information_gain, 5e-2

In [35]:
### TEST ###

# Checks the information gain for a split on the spambase dataset
t5_spambase_splits = split_on_numeric_value spambase["data"], "char_freq_$", 0.056
t5_spambase_entropy = entropy(class_distribution(spambase["data"]))
t5_spambase_information_gain = information_gain t5_spambase_entropy, t5_spambase_splits
assert_in_delta 0.17012249631509135, t5_spambase_information_gain, 5e-2

In [36]:
### TEST ###
sorted_values = iris["data"].collect {|r| r["features"]["petal_length"]}.uniq.sort
min_sl = sorted_values.first
max_sl = sorted_values.last

x = []
y = []
h0 = entropy(class_distribution(iris["data"]))
sorted_values.each do |t|
  x << t
  y << information_gain(h0, split_on_numeric_value(iris["data"], "petal_length", t))
end

assert_equal 43, sorted_values.size
Daru::DataFrame.new({x: x, y: y}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Petal Length Threshold"
  plot.y_label "Information Gain"
  plot.xrange [min_sl,max_sl]
end

#<Nyaplot::Plot:0x0000561e1f13e460 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000561e1f11cdd8 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"b03667cd-881e-4973-8f8b-deffc4d2fde1"}, @xrange=[1.0, 6.9], @yrange=[0.0, 0.6365141682948127]>], :options=>{:x_label=>"Petal Length Threshold", :y_label=>"Information Gain", :xrange=>[1.0, 6.9], :zoom=>true, :width=>700, :yrange=>[0.0, 0.6365141682948127]}}>

## Question 6 (10 points)
Finding the best split value by calling ```split_on_numeric_value``` on each threshold is _very_ expensive. There is a better solution. The function should return an array consisting of the best threshold and the information gain of that threshold.


In order to find the best split, we need to find $t^\star$ defined as follows:

$ t^\star = \arg \max_{t} IG(x,V(t)) $

Given that a threshold is a single real value, we can simply calculate the values $IG(x,V)$ for every value of $t$ _in order_, which leads allows us to save some work by calculating information incrementally. Notice that the split sets have a specialized form, which we can write as $V(t) = \left\{v_L(t), v_R(t)\right\}$ with $v_L(t) = \left\{x \mid x < t \right\}$ and $v_R(t) = \left\{x \mid x \ge t \right\}$. 

As we sweep $t$ from minimum to maximum values, we note the following relations:

$v_R(t + \Delta_t) = v_R(t) - v_\Delta$

$v_L(t + \Delta_t) = v_L(t) \cup v_\Delta$

where $v_\Delta = \left\{x \mid t < x \le t + \Delta_t \right\}$

What we add to $v_L$ is exactly what was removed from $v_R$, which means that we can easily recalculate the information gain by incrementing a class-specific counter for $v_L$ and decrementing a class-specific counter for $v_R$ as we sweep through the range of the threshold.

The function below returns an array with the split point and the information gain:
```ruby
return [t_max, ig_max]
```


In [37]:
def frequency g,na
  i = 0
  h = Hash.new()
  
  while i< na.length do
    sum = g.select { |k| k["label"] ==na[i]} 
    if sum.empty?      
      h.store(na[i],0)
    else
      h.store(na[i],sum.length)
    end
    i+=1
  end
  return h
end

:frequency

In [38]:
def find_split_point_numeric x, h0, fname
  # BEGIN YOUR CODE
y = x.collect {|r| r["features"][fname] }.uniq

sorted_values = y.reject { |c| c.nil? }.sort
g = x.group_by { |k| k["features"].include? fname }
v_L = g[false]
v_R = g[true].sort_by { |k| k["features"][fname] }

splits = {"l"=>v_L, "r"=>v_R}
name = ["l","r"]
i = 0

prob = Hash.new()
while i<splits.length do
  a = class_distribution(splits[name[i]])
  prob.store(name[i], a.each { |k, v| a[k] = v*(splits[name[i]].length) } )
  i+=1
end



na =  x.collect {|r| r["label"] }.uniq

i_g =[]
number = 0
sorted_values.each do |o|
  splits =  Hash.new()
  g = v_R.select { |k| k["features"][fname] <o} 
  v_L = v_L + g
  v_R = v_R - g
  splits = {"l"=>v_L, "r"=>v_R}

  
  #####update prob via g_hash


  g_hash = frequency g,na

  if g_hash.empty?
     w = prob["l"].clone()    
     g_hash = w.each {|k,v| w[k] = w[k]*0}
     number +=1
  end
  
  a = prob["l"].clone()
  b = prob["r"].clone()
  prob["l"] = a.merge(g_hash){ |k, a_value, g_value| a_value + g_value  }
  prob["r"] = b.merge(g_hash){ |k, b_value, g_value| b_value - g_value  }  



  
  i = 0
  sum =0.0
  h = 0.0
  while i<splits.length do
    a = prob[name[i]].clone()
    h += (splits[name[i]].length) * entropy( a.each { |k, v| a[k] = v/(splits[name[i]].length) } )
    sum += splits[name[i]].length
    i+=1
  end

  i_g.append(h0- h/sum)
  

end 
index = i_g.each_with_index.max[1]
return [sorted_values[index],i_g[index]]  
  
  #END YOUR CODE
end

:find_split_point_numeric

In [39]:
def find_split_point_numeric x, h0, fname
  # BEGIN YOUR CODE
y = x.collect {|r| r["features"][fname] }.uniq

sorted_values = y.reject { |c| c.nil? }.sort
g = x.group_by { |k| k["features"].include? fname }
v_L = g[false]
v_R = g[true].sort_by { |k| k["features"][fname] }
  

  
  
  
  
  
  
splits = {"l"=>v_L, "r"=>v_R}
name = ["l","r"]
i = 0

prob = Hash.new()
while i<splits.length do
  a = class_distribution(splits[name[i]])
  prob.store(name[i], a.each { |k, v| a[k] = v*(splits[name[i]].length) } )
  i+=1
end



na =  x.collect {|r| r["label"] }.uniq

i_g =[]
number = 0
sorted_values.each do |o|
  splits =  Hash.new()
  g = v_R.select { |k| k["features"][fname] <o} 
  v_L = v_L + g
  v_R = v_R - g
  splits = {"l"=>v_L, "r"=>v_R}

  
  #####update prob via g_hash


  g_hash = frequency g,na

  if g_hash.empty?
     w = prob["l"].clone()    
     g_hash = w.each {|k,v| w[k] = w[k]*0}
     number +=1
  end
  
  a = prob["l"].clone()
  b = prob["r"].clone()
  prob["l"] = a.merge(g_hash){ |k, a_value, g_value| a_value + g_value  }
  prob["r"] = b.merge(g_hash){ |k, b_value, g_value| b_value - g_value  }  



  
  i = 0
  sum =0.0
  h = 0.0
  while i<splits.length do
    a = prob[name[i]].clone()
    h += (splits[name[i]].length) * entropy( a.each { |k, v| a[k] = v/(splits[name[i]].length) } )
    sum += splits[name[i]].length
    i+=1
  end

  i_g.append(h0- h/sum)
  

end 
index = i_g.each_with_index.max[1]
return [sorted_values[index],i_g[index]]  
  
  #END YOUR CODE
end

:find_split_point_numeric

In [ ]:
 x = spambase["data"]
h0 = entropy(class_distribution(spambase["data"]))
fname = "char_freq_$"
#######################################################
  # BEGIN YOUR CODE
y = x.collect {|r| r["features"][fname] }.uniq
sorted_values = y.reject { |c| c.nil? }.sort




gg = x.group_by{ |k| k["features"].include? fname}

g = gg[true].select{ |k| k["features"][fname] < sorted_values[0]}
v_R = g[false]
v_L = g[true]  + gg[false]
puts "left",v_L,

splits = {"l"=>v_L, "r"=>v_R}
name = ["l","r"]
i = 0

prob = Hash.new()
while i<splits.length do
  
  if splits[name[i]].empty?
    a = class_distribution(splits[name[i+1]])
    prob.store(name[i], a.each { |k, v| a[k] = v*0.0 } )
  else
    a = class_distribution(splits[name[i]])
    prob.store(name[i], a.each { |k, v| a[k] = v*(splits[name[i]].length) } )
  end
  i+=1
end
puts prob,"prob"


na =  x.collect {|r| r["label"] }.uniq

i_g =[]
number = 0

c = p sorted_values.drop(1)

c.each do |o|
  splits =  Hash.new()
  g = v_R.select { |k| k["features"][fname] <o} 

  v_R = v_R - g
  v_L = v_L + g
  splits = {"l"=>v_L, "r"=>v_R}

  
  #####update prob via g_hash


  g_hash = frequency g,na

  if g_hash.empty?
     w = prob["l"].clone()    
     g_hash = w.each {|k,v| w[k] = w[k]*0}
     number +=1
  end

  a = prob["l"].clone()
  b = prob["r"].clone()
  prob["l"] = a.merge(g_hash){ |k, a_value, g_value| a_value + g_value  }
  prob["r"] = b.merge(g_hash){ |k, b_value, g_value| b_value - g_value  }  



  
  i = 0
  sum =0.0
  h = 0.0
  while i<splits.length do
    a = prob[name[i]].clone()
    h += (splits[name[i]].length) * entropy( a.each { |k, v| a[k] = v/(splits[name[i]].length) } )

    sum += splits[name[i]].length
    i+=1
  end

  i_g.append(h0- h/sum)
  

end 
index = i_g.each_with_index.max[1]-1
puts [c[index],i_g[index]]  


TypeError: no implicit conversion of false into Integer

In [ ]:
### TEST ###
t6_iris_entropy = entropy(class_distribution(iris["data"]))
t6_split_sepal_width = find_split_point_numeric iris["data"], t6_iris_entropy, "sepal_width"
puts 3.4, t6_split_sepal_width[0], 1e-2

NoMethodError: undefined method `+' for nil:NilClass

In [ ]:
### TEST ###
t6_iris_entropy = entropy(class_distribution(iris["data"]))
t6_split_sepal_width = find_split_point_numeric iris["data"], t6_iris_entropy, "sepal_width"
assert_in_delta 3.4, t6_split_sepal_width[0], 1e-2

NoMethodError: undefined method `+' for nil:NilClass

In [ ]:
### TEST ###
assert_in_delta 0.18570201019349364, t6_split_sepal_width[1], 1e-2

NoMethodError: undefined method `[]' for nil:NilClass

In [44]:
### TEST ###
t6_spambase_entropy = entropy(class_distribution(spambase["data"]))
t6_split_dollar = find_split_point_numeric spambase["data"], t6_spambase_entropy, "char_freq_$"
assert_in_delta 0.056, t6_split_dollar[0], 1e-2

In [45]:
### TEST ###
assert_in_delta 0.17012249631509135, t6_split_dollar[1], 1e-2

In [ ]:
### TEST: Produces same result as split_on_numeric_value ###
t6_iris_expected_ig = information_gain(t6_iris_entropy, split_on_numeric_value(iris["data"], "sepal_width", 3.4))
assert_in_delta t6_iris_expected_ig, t6_split_sepal_width[1], 1e-2
t6_spambase_expected_ig = information_gain(t6_spambase_entropy, split_on_numeric_value(spambase["data"], "char_freq_$", 0.056))
assert_in_delta t6_spambase_expected_ig, t6_split_dollar[1], 1e-2

NoMethodError: undefined method `[]' for nil:NilClass